In [1]:
import numpy as np
import random
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

In [2]:
# load the dataset into train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


In [3]:
print("train data")
print(x_train.shape)
print(y_train.shape)
print("\n test data")
print(x_test.shape)
print(y_test.shape)

train data
(60000, 28, 28)
(60000,)

 test data
(10000, 28, 28)
(10000,)


In [4]:
# sample only 20k images for training
idx = np.random.randint(x_train.shape[0], size=20000) # sample 20k indices from 0-60,000
x_train = x_train[idx, :]
y_train = y_train[idx]
print(x_train.shape)
print(y_train.shape)

(20000, 28, 28)
(20000,)


In [6]:
# specify input dimensions of each image
img_rows, img_cols = 28, 28
input_shape = (img_rows, img_cols, 1)

# batch size, number of classes, epochs
batch_size = 128
num_classes = 10
epochs = 12

In [7]:
# reshape x_train and x_test
x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
print(x_train.shape)
print(x_test.shape)
# convert class labels (from digits) to one-hot encoded vectors
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
y_test = tf.keras.utils.to_categorical(y_test, num_classes)
print(y_train.shape)

(20000, 28, 28, 1)
(10000, 28, 28, 1)
(20000, 10)


In [8]:
# convert int to float
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# normalise
x_train /= 255
x_test /= 255

**We will build a network with:**

two convolutional layers having 32 and 64 filters respectively,
followed by a max pooling layer,
and then Flatten the output of the pooling layer to give us a long vector,
then add a fully connected Dense layer with 128 neurons, and finally
add a softmax layer with 10 neurons

In [9]:
# model
model = Sequential()

# a keras convolutional layer is called Conv2D
# help(Conv2D)
# note that the first layer needs to be told the input shape explicitly

# first conv layer
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape)) # input shape = (img_rows, img_cols, 1)

# second conv layer
model.add(Conv2D(64, kernel_size=(3, 3),
                 activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
#Flatten and put a fully connected network
model.add(Flatten())
model.add(Dense(128, activation ='relu'))
model.add(Dropout(0.5))
# softmax layer
model.add(Dense(num_classes, activation='softmax'))
# model summary
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2  (None, 12, 12, 64)        0         
 D)                                                              
                                                                 
 dropout (Dropout)           (None, 12, 12, 64)        0         
                                                                 
 flatten (Flatten)           (None, 9216)              0         
                                                                 
 dense (Dense)               (None, 128)               1179776   
                                                        

Layer-1 (Conv2D): We have used 32 kernels of size (3, 3), and each kernel has a single bias, so we have 32 x 3 x 3 (weights) + 32 (biases) = 320 parameters (all trainable). Note that the kernels have only one channel since the input images are 2D (grayscale). By default, a convolutional layer uses stride of 1 and no padding, so the output from this layer is of shape 26 x 26 x 32, as shown in the summary above (the first element None is for the batch size).

Layer-2 (Conv2D): We have used 64 kernels of size (3, 3), but this time, each kernel has to convolve a tensor of size (26, 26, 32) from the previous layer. Thus, the kernels will also have 32 channels, and so the shape of each kernel is (3, 3, 32) (and we have 64 of them). So we have 64 x 3 x 3 x 32 (weights) + 64 (biases) = 18496 parameters (all trainable). The output shape is (24, 24, 64) since each kernel produces a (24, 24) feature map.

Max pooling: The pooling layer gets the (24, 24, 64) input from the previous conv layer and produces a (12, 12, 64) output (the default pooling uses stride of 2). There are no trainable parameters in the pooling layer.

The Dropout layer does not alter the output shape and has no trainable parameters.

The Flatten layer simply takes in the (12, 12, 64) output from the previous layer and 'flattens' it into a vector of length 12 x 12 x 64 = 9216.

The Dense layer is a plain fully connected layer with 128 neurons. It takes the 9216-dimensional output vector from the previous layer (layer l-1) as the input and has 128 x 9216 (weights) + 128 (biases) = 1179776 trainable parameters. The output of this layer is a 128-dimensional vector.

The Dropout layer simply drops a few neurons.

Finally, we have a Dense softmax layer with 10 neurons which takes the 128-dimensional vector from the previous layer as input. It has 128 x 10 (weights) + 10 (biases) = 1290 trainable parameters.

Thus, the total number of parameters are 1,199,882 all of which are trainable.

In [10]:
model.compile(loss=tf.keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [11]:
# fit the model

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

Epoch 1/12
157/157 [==============================] - 62s 390ms/step - loss: 2.3030 - accuracy: 0.1135 - val_loss: 2.2941 - val_accuracy: 0.1520
Epoch 2/12
157/157 [==============================] - 56s 358ms/step - loss: 2.2894 - accuracy: 0.1309 - val_loss: 2.2794 - val_accuracy: 0.1781
Epoch 3/12
157/157 [==============================] - 56s 356ms/step - loss: 2.2761 - accuracy: 0.1580 - val_loss: 2.2639 - val_accuracy: 0.2066
Epoch 4/12
157/157 [==============================] - 58s 368ms/step - loss: 2.2608 - accuracy: 0.1874 - val_loss: 2.2474 - val_accuracy: 0.2472
Epoch 5/12
157/157 [==============================] - 56s 357ms/step - loss: 2.2463 - accuracy: 0.2086 - val_loss: 2.2296 - val_accuracy: 0.3038
Epoch 6/12
157/157 [==============================] - 56s 355ms/step - loss: 2.2295 - accuracy: 0.2397 - val_loss: 2.2103 - val_accuracy: 0.3517
Epoch 7/12
157/157 [==============================] - 60s 384ms/step - loss: 2.2096 - accuracy: 0.2669 - val_loss: 2.1890 - val_ac

In [12]:
# evaluate the model on test data
model.evaluate(x_test, y_test)

313/313 [==============================] - 8s 24ms/step - loss: 2.0410 - accuracy: 0.5875


[2.041016101837158, 0.5874999761581421]

In [13]:
print(model.metrics_names)

['loss', 'accuracy']
